<a href="https://colab.research.google.com/github/nivin18/RealTimeMLProjects/blob/master/FAKE_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading Data from Kaggle

In [1]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nivinabrahamthomas","key":"1b2fe58e372955284fffd4d87be5b08c"}'}

In [2]:
! pip install -q kaggle

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
! mkdir ~/.kaggle
!cp /content/gdrive/My\ Drive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c fake-news --force

 97% 45.0M/46.5M [00:00<00:00, 138MB/s]
100% 46.5M/46.5M [00:00<00:00, 128MB/s]


In [6]:
! unzip fake-news.zip



Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


Importing data

In [8]:
import pandas as pd
data=pd.read_csv("train.csv")
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


Removing Nan Values from the dataset

In [9]:
data = data.dropna()
data.reset_index(inplace=True)

Extracting Dependent and Independent variables from data

In [10]:
x = data.drop("label", axis = 1)
y = data["label"]

Checking the Shape of data

In [11]:
print(x.shape)
print(y.shape)

(18285, 5)
(18285,)


Importing the required Libraries

In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import random
import json


Preprocessing the Text

In [13]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
messages=x.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a=zA-Z]"," ", messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # (word is not n stopwords) and stem(word)
  review = ' '.join(review)
  corpus.append(review)

one Hot Encoding

In [39]:
vocab_size = 5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus]
onehot_repr

[[2111, 4267, 2463, 2990, 4097, 2992, 3987, 2990, 2341],
 [158, 2111, 2990, 162, 4267, 3923, 162],
 [4267, 1805, 1370],
 [2990, 4066, 4700, 2189, 162],
 [],
 [3987, 2111, 4267, 4097, 2111, 162, 1143, 4066, 1273, 1386, 2463, 2112, 162],
 [162, 2189, 4267, 1370, 1223, 3394, 1143, 1143],
 [1386, 2990, 3394, 2992, 1707, 2990, 1143],
 [2341, 1223, 4097],
 [3907, 2990, 1707, 2111, 2751, 3923],
 [1707, 1707, 2576, 1386, 2111, 3394],
 [4700, 1143, 4097, 162, 1707],
 [1707, 1223, 1805, 162, 1370, 4267],
 [4097, 1370, 2111, 1143],
 [4267, 835, 2990, 1143],
 [1223, 1707, 2463, 4009, 3394],
 [1929, 2990, 2111],
 [2990, 162, 2977, 3394, 835, 162],
 [4097, 2990, 2990],
 [1707, 1707, 2992, 162],
 [3394, 1707, 1273, 1959, 2341, 1143],
 [1143, 4335, 2111, 1386, 1143, 4267],
 [1796, 2112, 4267, 1370, 162],
 [2111, 2111, 2111, 4267, 3923, 3923, 162, 4731],
 [1392, 1370, 1143, 2990, 1707, 2189, 2990, 1143],
 [2111, 2111, 2990, 162, 2189, 1143],
 [2990, 162, 2977, 2297, 4535, 3394, 835, 1143, 162],
 [2189,

Applying pad Sequences to make Sentence lenth

In [40]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3987 2990 2341]
 [   0    0    0 ... 4267 3923  162]
 [   0    0    0 ... 4267 1805 1370]
 ...
 [   0    0    0 ... 2111 1386 1143]
 [   0    0    0 ... 1223 2463 1386]
 [   0    0    0 ... 4267 4066 1370]]


Creating Model for Train set

In [41]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [42]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size = 0.33, random_state = 42)

In [44]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.4515 - accuracy: 0.7907 - val_loss: 0.3291 - val_accuracy: 0.8747
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.3179 - accuracy: 0.8784 - val_loss: 0.3257 - val_accuracy: 0.8769
Epoch 3/10
192/192 [==============================] - 7s 39ms/step - loss: 0.3082 - accuracy: 0.8819 - val_loss: 0.3169 - val_accuracy: 0.8759
Epoch 4/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3032 - accuracy: 0.8811 - val_loss: 0.3138 - val_accuracy: 0.8784
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.2948 - accuracy: 0.8837 - val_loss: 0.3126 - val_accuracy: 0.8779
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.2872 - accuracy: 0.8847 - val_loss: 0.3083 - val_accuracy: 0.8774
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.2801 - accuracy: 0.8887 - val_loss: 0.3037 - val_accuracy: 0.8800

Performance Metrics and Accuracy

In [48]:
y_pred = model.predict(x_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

189/189 [==============================] - 1s 6ms/step


array([[2953,  466],
       [ 281, 2335]])

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8762220381110191